In [1]:
import time
import matplotlib.pyplot as plt
import numpy as np

from DataAnalysis import DataAnalysis
#%run new_DataAnalysis.ipynb|
from MonteCarlo import MonteCarlo

In [2]:
# data = DataAnalysis()
# data.load_eventdf('data/new_moo/8-5/events.csv')
# data.load_clusterdf('data/new_moo/8-5/clusters.csv')
# data.filter_fit(3.5, 5) #(NRMSE, channels)
# data.get_cluster(0)

In [3]:
# rand_times = np.sort(np.random.choice(np.arange(1000),10,replace=False, dtype=np.float64))
# rand_times[0]

In [4]:
mc = MonteCarlo(num_tracks=5, track_type="sasso", particle_type="muon")
#mc.filter_fit(100, 3)
#mc.get_eventdf()

# mc = DataAnalysis()
# mc.load_eventdf('data/new_moo/3-30/MC_events.csv')
# mc.load_clusterdf('data/new_moo/3-30/MC_clusters.csv')

[15949. 17184. 25665. 26502. 42715.]


In [8]:
mc.get_clusterdf()

,Cluster,Events,Channels,StartTime,TimeSpread,NRMSE,Zenith,Azimuth,ExtraCh,MissingCh,Likelihood,Fitline
0,0,7,7,15952.2,0.06,0.910884,0.911280,0.641559,0,0,-19.007828,"[89.032118, -261.838038, -2.371231, -0.231076,..."
1,1,6,6,17187.2,0.05,1.018935,0.336453,-0.365137,0,0,-15.315252,"[-139.924858, 7.695506, -198.897636, 0.318411,..."
2,2,3,3,25668.2,0.02,1.374818,0.102074,1.403117,0,0,-9.129201,"[-186.886024, 128.776252, 240.32089, 0.045677,..."
3,3,4,4,26505.2,0.03,1.427791,0.954891,-0.581643,0,0,-13.973394,"[-17.196762, 338.452059, -107.788938, -0.81519..."
4,4,9,9,42718.2,0.08,0.960109,0.951575,0.640503,0,0,-23.105973,"[211.157086, 222.116282, -118.809238, -0.23892..."


In [6]:
# from pymoo.visualization.scatter import Scatter
# test = DataAnalysis()
# line, pre_result, fin_result = test.fitline(mc.get_cluster(0), pop_num=50, gen_num=50)

In [7]:
# plot = Scatter(title = "Objective Space", angle=(0,45))
# plot.add(pre_result.F)
# plot.show()

In [ ]:
# plot = Scatter(title = "Objective Space", angle=(0,45))
# plot.add(fin_result.F[:,:2])
# plot.show()

In [ ]:
start = time.time()

test = DataAnalysis()

test.set_eventdf(mc.get_cluster(range(5)))
#test.set_eventdf(mc.get_cluster(1))

#test.arrange_clusters(3, 1.0) # >= 5 events, <= 1.0 seconds
test.make_clusterdf(pop_num=100, gen_num=100, verbose=True)

#test.save_eventdf("./data/test_eventdf.csv")
#test.save_clusterdf("./data/test_clusterdf.csv")

time.time() - start

In [ ]:
plt.figure(figsize=(5,5))
plt.title("Pathlengths")
plt.scatter( np.array(mc.get_cluster(range(5))['PathLength']), np.array(test.get_eventdf()['PathLength']), s=5)
plt.ylabel("mm (reconstructed)")
plt.xlabel("mm (generated)")
plt.xlim(0,80)
plt.ylim(0,80)

In [ ]:
from Helper import *

dEdx_pdf = make_pdf('./data/pdf/dEdx/bins_dEdx_muon.csv', './data/pdf/dEdx/values_dEdx_muon.csv', domain_range=[0,190])
clusters = np.array(mc.get_clusterdf()['Cluster'])

log_likelihoods_mc = [np.log(dEdx_pdf(np.array(mc.get_cluster(c)['dEdx'])/100)).sum() for c in clusters]
log_likelihoods_test = [np.log(dEdx_pdf(np.array(test.get_cluster(c)['dEdx'])/100)).sum() for c in clusters]


# min_path_test = test.get_eventdf()[test.get_eventdf()['PathLength'] > 30]
# min_path_mc = mc.get_eventdf()[test.get_eventdf()['PathLength'] > 30]

plt.figure(figsize=(12,8))

plt.subplot(2,3,1)
plt.title("dEdx")
#plt.scatter(np.array(mc.get_eventdf()['dEdx'])/100, np.array(test.get_eventdf()['dEdx'])/100, s=5)
plt.scatter(test.get_eventdf()['dEdx']/100, mc.get_eventdf()['dEdx']/100, s=5)
plt.xlabel("MeV/cm (reconstructed)")
plt.ylabel("MeV/cm (generated)")
plt.xlim(4,30)
plt.ylim(4,30)
plt.xscale('log')
plt.yscale('log')

plt.subplot(2,3,2)
plt.title("dEdx log-likelihoods")
plt.scatter(log_likelihoods_mc, log_likelihoods_test, s=5)
plt.xlim(-100,0)
plt.ylim(-100,0)
plt.xlabel("log(likelihood) (generated)")
plt.ylabel("log(likelihood) (reconstructed)")

plt.subplot(2,3,3)
plt.title("Pathlengths")
plt.scatter( np.array(mc.get_eventdf()['PathLength']), np.array(test.get_eventdf()['PathLength']), s=5)
plt.ylabel("mm (reconstructed)")
plt.xlabel("mm (generated)")
plt.xlim(0,80)
plt.ylim(0,80)

plt.subplot(2,3,4)
plt.title("Zenith")
plt.scatter(np.array(mc.get_clusterdf()['Zenith']) * 180 / np.pi, np.array(test.get_clusterdf()['Zenith']) * 180 / np.pi, s=5)
plt.xlabel("degrees (reconstructed)")
plt.ylabel("degrees (generated)")


plt.subplot(2,3,5)
plt.title("Azimuth")
plt.scatter(np.array(mc.get_clusterdf()['Azimuth']) * 180 / np.pi, np.array(test.get_clusterdf()['Azimuth']) * 180 / np.pi, s=5)
plt.xlabel("degrees (reconstructed)")
plt.ylabel("degrees (generated)")

plt.tight_layout()

In [ ]:
test.get_clusterdf(fitline=False)

In [ ]:
mc.get_clusterdf(fitline=False)